### Test Offside logic on labelled data

In [3]:
import cv2, json, numpy as np

# 1. Load image & annotations
img_path = "6.png"     # change to "1.png", "8.png", etc.
# img_path = "9_0.png" 
frame = cv2.imread(img_path)
h, w  = frame.shape[:2]
with open("Labels-v3.json","r") as f:
    ann = json.load(f)["actions"][img_path]
    # ann = json.load(f)["replays"][img_path]

# 2. Find which side-line exists: left or right
line_classes = {l["class"] for l in ann["lines"]}

# priority: Side line -> Big rect. -> Small rect.
if "Side line right" in line_classes:
    touch_cls = "Side line right"
elif "Side line left" in line_classes:
    touch_cls = "Side line left"
elif "Big rect. right main" in line_classes:
    touch_cls = "Big rect. right main"
elif "Big rect. left main" in line_classes:
    touch_cls = "Big rect. left main"
elif "Small rect. right main" in line_classes:
    touch_cls = "Small rect. right main"
elif "Small rect. left main" in line_classes:
    touch_cls = "Small rect. left main"
else:
    raise ValueError("No usable vertical field-edge found in annotations")

# Always prefer side line top as the horizontal reference
if   "Side line top"    in line_classes: horizontal_cls = "Side line top"
elif "Side line bottom" in line_classes: horizontal_cls = "Side line bottom"
elif "Big rect. left bottom" in line_classes: horizontal_cls = "Big rect. left bottom"
elif "Big rect. left top" in line_classes: horizontal_cls = "Big rect. left top"
else:
    raise RuntimeError("No horizontal pitch edge found in annotations")

# Helper: get two endpoints along major axis
def endpoints(cls, axis):
    ln = next(l for l in ann["lines"] if l["class"]==cls)
    pts = np.array(ln["points"],float).reshape(-1,2)
    if axis=='x':
        i0,i1 = pts[:,0].argmin(), pts[:,0].argmax()
    else:
        i0,i1 = pts[:,1].argmin(), pts[:,1].argmax()
    return pts[i0], pts[i1]

# Get endpoints of a top and side line
A_top,  B_top  = endpoints(horizontal_cls, 'x')
print(A_top, B_top)
A_side, B_side = endpoints(touch_cls,       'y')
print(A_side, B_side)

# Draw references
cv2.line(frame, tuple(A_top.astype(int)),  tuple(B_top.astype(int)),  (200,200,255),2)
cv2.line(frame, tuple(A_side.astype(int)), tuple(B_side.astype(int)), (200,200,255),2)

# 3. Extend touch-line to bottom or top so it spans full frame
d_line = B_side - A_side
if abs(d_line[1])>1e-6:
    # Extend downward if right touch-line, upward if left
    target_y = h if touch_cls.endswith("right") else 0
    t = (target_y - A_side[1]) / d_line[1]
    B_ext = A_side + d_line * t
else:
    B_ext = B_side.copy()
cv2.line(frame, tuple(A_side.astype(int)), tuple(B_ext.astype(int)), (200,200,255),2)

# 4. Pick defender & attacker by perpendicular distance to goal line or along a unit vector parallel to side line
def pick_closest(team_cls, color):
    # Original side‐line vector & its length
    side_vec   = B_side - A_side
    side_len   = np.linalg.norm(side_vec)

    print(side_len)
    print(h)

    # Unit‐vector along top‐line for fallback
    u_top = (B_top - A_top)
    u_top /= np.linalg.norm(u_top)

    # A threshold on the raw annotation length
    min_len = 0.5 * h   # e.g. 50% of frame height

    best = None
    for bb in ann["bboxes"]:
        if bb["class"] != team_cls:
            continue

        # Choose correct foot pixel (according to the side of the pitch)
        if touch_cls.endswith("right"):
            fx = bb["points"]["x2"]
        else:
            fx = bb["points"]["x1"]
        fy = bb["points"]["y2"]
        P  = np.array([fx, fy], dtype=float)

        # If the annotated side‐line is long enough, use the true perp‐distance
        if side_len > min_len:
            dist = abs(np.cross(side_vec, P - A_side)) / side_len
        else:
            # Fallback: project along the side‐line direction
            dist = abs((P - A_side).dot(u_top))

        if best is None or dist < best[0]:
            best = (dist, P, (bb["points"]["x1"],bb["points"]["y1"],
                              bb["points"]["x2"],bb["points"]["y2"]))

    # Draw the chosen box & foot
    d, foot, (x1,y1,x2,y2) = best
    cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),color,2)
    cv2.circle(   frame,(int(foot[0]),int(foot[1])),6,color,-1)

    return best

# Defender = same-side team as touch-line (if right touch-line, defender team right)
def_team = "Player team right" if touch_cls=="Side line right" else "Player team left"
att_team = "Player team left"  if touch_cls=="Side line right" else "Player team right"
dist_def, def_foot, _ = pick_closest(def_team,(0,255,0))
dist_att, att_foot, _ = pick_closest(att_team,(0,255,255))

# 5. Compute vanishing point V for “vertical” references
vert_classes = {touch_cls, 
                "Big rect. right main" if touch_cls.endswith("right") else "Big rect. left main",
                "Small rect. right main" if touch_cls.endswith("right") else "Small rect. left main"}
lines = []
for ln in ann["lines"]:
    if ln["class"] in vert_classes:
        pts = np.array(ln["points"],float).reshape(-1,2)
        A,B = pts[pts[:,1].argmin()], pts[pts[:,1].argmax()]
        lines.append(np.cross(np.append(A,1), np.append(B,1)))
L = np.stack(lines,axis=0)
_,_,vh = np.linalg.svd(L)
V = vh[-1]; V/=V[2]
vx, vy = V[0], V[1]

# 6. Draw offside lines (parallel to goal-line) through each foot
Lspan = max(h,w)*1.5
for foot, col in [(def_foot,(0,0,255)), (att_foot,(255,0,255))]:
    dir_img = np.array([vx,vy]) - foot
    dir_img /= np.linalg.norm(dir_img)
    P1 = foot - dir_img*Lspan
    P2 = foot + dir_img*Lspan
    cv2.line(frame, tuple(P1.astype(int)), tuple(P2.astype(int)), col,3)

# 7. OFFSIDE decision
if dist_att < dist_def:
    cv2.putText(frame,"OFFSIDE",(50,80),
                cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

# 8. Show result
cv2.namedWindow("Offside Unified", cv2.WINDOW_NORMAL)
cv2.imshow("Offside Unified", frame)
cv2.resizeWindow("Offside Unified",960,int(960*h/w))
cv2.waitKey(0)
cv2.destroyAllWindows()


[  0.   326.33] [975.55 277.77]
[967.78 277.77] [1920.    514.75]
981.2657381158277
1080
981.2657381158277
1080
